## Tabular Data

### Tidy Data from Excel

An Excel spreadsheet with some brief information on awards given to movies is available at:

> https://www.gnosis.cx/cleaning/Film_Awards.xlsx

In a more fleshed out case, we might have data for many more years, more types of awards, more associations that grant awards, and so on.  While the organization of this spreadsheet is much like a great many you will encounter "in the wild," it is very little like the tidy data we would rather work with.  In the simple example, only 63 data values occur, and you could probably enter them into the desired structure by hand as quickly as coding the transformations.  However, the point of this exercise is to write programming code that could generalize to larger data sets of similar structure.

<img src="img/Film_Awards.png" alt="Film Awards"/>

__Image: Film Awards Spreadsheet__

Your task in this exercise is to read this data into a single well-normalized data frame, using whichever language and library you are most comfortable with.  Along the way, you will need to remediate whatever data integrity problems you detect.  As examples of issues to look out for:

* The film _1917_ was stored as a number not a string when naïvely entered into a cell.
* The spelling of some values is inconsistent.  Olivia Colman's name is incorrectly transcribed as 'Coleman' in one occurrence.  There is a spacing issue in one value you will need to identify.
* Structurally, an apparent parallel is not really so.  Person names are sometimes listed under the name of the association, but elsewhere under another column.  Film names are sometimes listed under association, other times elsewhere.
* Some column names occur multiple times in the same tabular area.

In thinking about a good data frame organization, think of what the independent and dependent variables are.  In each year, each association awards for each category. These are independent dimensions.  A person name and a film name are slightly tricky since they are not exactly independent, but at the same time some awards are to a film and others to a person.  Moreover, one actor might appear in multiple films in a year (not in this sample data; but do not rule it out).  Likewise, at times multiple films have used the same name at times in film history. Some persons are both director and actor (in either the same or different films).

Once you have a useful data frame, use it to answer these questions in summary reports:

* For each film involved in multiple awards, list the award and year it is associated with.
* For each actor/actress winning multiple awards, list the film and award they are associated with.
* While not occurring in this small data set, sometimes actors/actresses win awards for multiple films (usually in different years).  Make sure your code will handle that situation.
* It is manual work, but you may want to research and add awards given in other years; in particular, adding some data will show actors with awards for multiple films.  Do your other reports correctly summarize the larger data set?

### Tidy Data from SQL

An SQLite database with roughly the same brief information as in the prior spreadsheet is available at:

> https://www.gnosis.cx/cleaning/Film_Awards.sqlite

However, the information in the database version is relatively well normalized and typed.  Also, additional information has been included on a variety of entities included in the spreadsheet.  Only slightly more information is included in this schema than in the spreadsheet, but it should be able to accommodate a large amount of data on films, actors, directors, and awards, and the relationships among those data.

```sql
sqlite> .tables
actor     award     director  org_name
```

As was mentioned in the prior exercise, the same name for a film can be used more than once, even by the same director.  For example  Abel Gance, used the title _J'accuse!_ for both his 1919 and 1938 films with connected subject matter.

```
sqlite> SELECT * FROM director WHERE year < 1950;
Abel Gance|J'accuse!|1919
Abel Gance|J'accuse!|1938
```

Let us look at a selection from the `actor` table, for example.  In this table we have a column `gender` to differentiate beyond name. As of this writing, no transgender actor has been nominated for a major award both before and after a change in gender identity, but this schema allows for that possibility.  In any case, we can use this field to differentiate the "actor" versus "actress" awards that many organizations grant.

```sql
sqlite> .schema actor
CREATE TABLE actor (name TEXT, film TEXT, year INTEGER, gender CHAR(1));

sqlite> SELECT * FROM actor WHERE name="Joaquin Phoenix";
Joaquin Phoenix|Joker|2019|M
Joaquin Phoenix|Walk the Line|2006|M
Joaquin Phoenix|Hotel Rwanda|2004|M
Joaquin Phoenix|Her|2013|M
Joaquin Phoenix|The Master|2013|M
```

The goal in this exercise is to create the same tidy data frame that you created in the prior exercise, and answer the same questions that were asked there.  If some questions can be answered directly with SQL, feel free to take that approach instead.  For this exercise, only consider awards for the years 2017, 2018, and 2019.  Some others are included in an incomplete way, but your reports are for those years.

```sql
sqlite> SELECT * FROM award WHERE winner="Frances McDormand";
Oscar|Best Actress|2017|Frances McDormand
GG|Actress/Drama|2017|Frances McDormand
Oscar|Best Actress|1997|Frances McDormand
```

## Anomoly Detection

The two exercises in this chapter ask you to look for anomalies first in quantitative data, then in categorical data.

### A Famous Experiment

The Michelson–Morley experiment was an attempt in the late 19th century to detect the existence of the *luminiferous aether*, a widely assumed medium that would carry light waves.  This was the most famous "failed experiment" in the history of physics in that it did not detect what it was looking for—something we now know not to exist at all.  The general idea was to measure the speed of light under different orientations of the equipment relative to the direction of movement of the earth, since relative movement of the ether medium would add or subtract from the speed of the wave.  Yes, it does not work that way under the theory of relativity, but it was a reasonable guess 150 years ago.

Apart from the physics questions, the data set derived by the Michelson-Morley experiment is widely available, including as a sample built into R.  The same data is available at:

> https://www.gnosis.cx/cleaning/morley.dat

Figuring out the format, which is not complex, is a good first step of this exercise (and typical of real data science work).

The specific numbers in this data are measurements of the speed of light in km/s with a zero point of 299,000.  So, for example, the mean measurement in experiment 1 was 299,909 km/s.  Let us look at the data in the R bundle.

In [47]:
%%R -o morley
data(morley)
morley %>%
    group_by(`Expt`) %>%
    summarize(Mean = mean(Speed), Count = max(Run))

`summarise()` ungrouping output (override with `.groups` argument)
# A tibble: 5 x 3
   Expt  Mean Count
  <int> <dbl> <int>
1     1  909     20
2     2  856     20
3     3  845     20
4     4  820.    20
5     5  832.    20


In the summary, we just look at the number of runs of each experimental setup, and the mean across that setup.  The raw data has 20 measurements within each setup.

Using whatever programming language and tools you prefer, identify the outliers first within each setup (defined by an `Expt` number) and then within the data collection as a whole.  The hope in the original experiment was that each setup would show a significant difference in central tendency, and indeed their means are somewhat different.  This book and chapter does not explore confidence levels and null hypotheses in any detail, but create a visualization that aids you in gaining visual insight into how much apparent difference exists between the several setups.

If you discard the outliers within each setup, are the differences between setups increased or decreased? Answer with either a visualization or by looking at statistics on the reduced groups.

### Misspelled Words

For this exercise we return to the 25,000 human measurements we have used to illustrate a number of concepts.  However, in this variation of the data set, each row has a person's first name (pulled from the US Social Security Agency list of common first names over the last century; apologies that the names lean Anglocentric because of the past history of US population and immigration trends).

The data set for this exercise can be found at:

> https://www.gnosis.cx/cleaning/humans-names.csv

Unfortunately, our hypothetical data collectors for this data set are simply terrible typists, and they make typos when entering names with alarming frequency.  There are some number of intended names in this data set, but quite a few simple miscodings of those names as well.  The problem is: how do we tell a real name from a typo?

There are a number of ways to measure the similarity of strings, and that provide a clue as to likely typos.  One general class of approach is in terms of *edit distance* between strings. The R package **stringdist**, for example provides Damerau-Levenshtein, Hamming, Levenshtein, and optimal sting alignment, as measures of edit distance.  Less edit-specific fuzzy matching techniques utilize a "bag of n-grams" approach, and include q-gram, cosine distance, and Jaccard distance. Some heuristic metrics like Jaro and Jaro-Winkler are also included in `stringdist` along with the other measures mentioned.  Soundex, soundex variants, and metaphone look for similarity of the sounds of words as pronounced, but are therefore specific to language and even regional dialect.

In a reversal of the more common pattern of Python versus R libraries, Python is the one that scatters string similarity measures over numerous libraries, each including just a few measures.  However, **python-Levenshtein** is a very nice package including most of these measures.  If you want cosine similarity, you may have to use `sklearn.metrics.pairwise` or another module.  For phonetic comparisons, **fonetika** and **soundex** both support multiple languages (but different languages for each; English is in common for almost all packages).

On my personal system, I have a command-line utility called `similarity` that I use to measure how close strings are to each other.  This particular few line script measures Levenshtein distance, but also normalizes it to the length of the longer string.  A short name will have a small numeric measure of distance, even betweeen dissimilar strings, while long strings that are close overall can have a larger measure before normalization (depending on what measure is chosen, but for most of them).  A few examples show this.

In [48]:
%%bash 
similarity David Davin

Levenshtein distance: 1
Similarity ratio: 0.8


In [49]:
%%bash
similarity David Maven

Levenshtein distance: 3
Similarity ratio: 0.4


In [50]:
%%bash
similarity "the quick brown fox jumped" \
           "thee quikc brown fax jumbed"

Levenshtein distance: 5
Similarity ratio: 0.814814814815


For this exercise, your goal is to identify every *genuine* name, and correct all the misspelled ones to the correct canonical spelling.  Keep in mind that sometimes multiple legitimate names are actually close to each other in terms of similarity measures.  However, it is probably reasonable to assume that *rare* spellings are typos, at least if they are also relatively similar to common spellings.  You may use whatever programming language, library, and metric you feel is the most useful for the task.

Reading in the data, we see it is similar to the human measures we have seen before.

In [51]:
names = pd.read_csv('data/humans-names.csv')
names.head()

,Name,Height,Weight
0,James,167.089607,64.806216
1,David,181.648633,78.281527
2,Barbara,176.272800,87.767722
3,John,173.270164,81.635672
4,Michael,172.181037,82.760794


It is easy to see that some "names" occur very frequently and others only rarely.  Look at the middling values as well in working on this exercise.

In [52]:
names.Name.value_counts()

Elizabeth    1581
Barbara      1568
Jessica      1547
Jennifer     1534
             ... 
Josep           1
iWlliam         1
Joseeph         1
eJennifer       1
Name: Name, Length: 249, dtype: int64